In [1]:
import sys
sys.path.append('../../Facial-Landmarks-Detector/')
sys.path.append('../../Revelio-LipsMovement/')
sys.path.append('../../Dynamic-texture-analysis-for-detecting-face-faces-in-video-sequences/')
sys.path.append('../../Modules/dft/lib/')

In [2]:
import cv2
from matplotlib import pyplot as plt
import dlib
from landmarks_detector import *
from LipMovementClassifier import *
from FDA_Model import *
import numpy as np

c:\Users\ammar\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
#read video
cap = cv2.VideoCapture('002_007.mp4')

#read frames into numpy array
grayFrames = []
while True:
    ret, frame = cap.read()
    if ret:
        #frame to gray scale
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        grayFrames.append(frame)
    else:
        break



In [4]:
#crop face from frames
faceFrames = []
detector = dlib.get_frontal_face_detector()
for frame in grayFrames:
    face = detector(frame)[0]
    x1 = face.left()
    y1 = face.top()
    x2 = face.right()
    y2 = face.bottom()
    faceFrames.append(frame[y1:y2, x1:x2])

In [5]:
#resize face frames to the minimum frace size
minSize = min([face.shape[0] for face in faceFrames])
faceFrames = [cv2.resize(face, (minSize, minSize)) for face in faceFrames]

In [6]:
landmarksDetector = LandmarksDetector(isPredictor=True, modelspath='../../Facial-Landmarks-Detector/landmarksmodels')

In [7]:
framesLandmarks = []
for frame in faceFrames:
    framesLandmarks.append(landmarksDetector.predict(frame, (0, 0, frame.shape[1], frame.shape[0])))
    

In [8]:
LipsModelsLocations = '../../Revelio-LipsMovement/trainedmodels/'
LipModelsPaths = [LipsModelsLocations + 'deepfakes/lips_movements_classifer.pth', LipsModelsLocations + 'face2face/lips_movements_classifer.pth', LipsModelsLocations + 'faceswap/lips_movements_classifer.pth', LipsModelsLocations + 'neuraltextures/lips_movements_classifer.pth']


In [9]:
faceFrames = np.array(faceFrames)
framesLandmarks = np.array(framesLandmarks)


In [10]:
lipsMovementModel = LipMovementClassifier(isPredictor=True, predictionMSTCNModelPaths=LipModelsPaths, featureExtractorModelPath='../../Revelio-LipsMovement/trainedmodels/resnet_feature_extractor.pth', mstcnConfigFilePath='../../Revelio-LipsMovement/models/configs/mstcn.json')

In [11]:
allLipsPredictions = lipsMovementModel.predict(faceFrames, framesLandmarks)

In [12]:
np.average(allLipsPredictions[0])

0.5812853

In [13]:
np.average(allLipsPredictions[1])

0.16849901

In [14]:
np.average(allLipsPredictions[2])

0.9991908

In [15]:
np.average(allLipsPredictions[3])

0.75983006

In [16]:
fdaModel = FDA(model_path='../../Modules/dft/Models')

c:\Users\ammar\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 1.2.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [17]:
fdaResults = fdaModel.predict(faceFrames)

In [18]:
fdaResults

{'deepfake': 0, 'face2face': 0, 'faceswap': 0, 'neuraltextures': 0}